In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

In [3]:
''' initializing weights and bias '''

def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def init_bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

In [4]:
'''defining the convolutional network 2d architecture'''

def conv2d(x,W):
    # x --> [batch, H, W, channels]
    # W --> [filter H, filter W, channels In, channles Out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

In [5]:
''' Pooling '''

def max_pool(x): 
    # --> ofcourse the same pool as its just an image to be filtered and return
    
    return tf.nn.max_pool(x,ksize=[1,2,2,1], padding='SAME', strides=[1,2,2,1]) # is where I want to do the pooling
    # i.e. with H and W, not on batch and channel 

In [6]:
''' convolutional and deep nn layer helper funcitons '''

def convlayer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    
    return tf.nn.relu(conv2d(input_x,W)+b)

def normalayer(input_layer, size):
    inp_size = int(input_layer.get_shape()[1])
    W = init_weights([inp_size, size])
    b = init_bias([size])
    
    return tf.matmul(input_layer,W)+b



In [7]:
'''place holders'''

x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None, 10])

In [8]:
# reshaping the image

x_image = tf.reshape(x,[-1,28,28,1])

# building layers

In [9]:
''' CNN architecture '''

conv_1 = convlayer(x_image,shape=[5,5,1,32])
conv_1_pool = max_pool(conv_1)

conv_2 = convlayer(conv_1_pool, shape=[5,5,32,64])
conv_2_pool = max_pool(conv_2)

conv_2_flat = tf.reshape(conv_2_pool, shape=[-1,7*7*64])
full_layer = tf.nn.relu(normalayer(conv_2_flat, 1024))

In [10]:
''' With dropout '''

hold_prob = tf.placeholder(tf.float32)
full_drop_out = tf.nn.dropout(full_layer,keep_prob=hold_prob)

In [11]:
''' predictions '''

y_pred = normalayer(full_drop_out,10)

In [12]:
# LOSS FUNCTION

cross = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [13]:
output = tf.train.AdamOptimizer(learning_rate=0.001)
train = output.minimize(cross)

In [14]:
''' Training Session '''

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(1000):
        bx, by = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict={x:bx, y_true:by,hold_prob:0.5})
        
        if i%100==0:
            print("accuracy ")
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels, hold_prob:1.0}))